In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:

filename_queue = tf.train.string_input_producer(["sample.csv"], num_epochs=1)

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
win_len = 3
seq_len = (win_len * 2) + 1
num_feats = 1
num_cols = seq_len + (seq_len * num_feats) + 1 # num tokens plus num feats per token plus label
record_defaults = [['<UNK>']] * num_cols


line = tf.decode_csv( value, record_defaults=record_defaults)
tokens = tf.stack(line[:seq_len])

features = tf.expand_dims(tf.stack(line[seq_len:-1]), 1)
label = line[-1]

batch_size = 4
min_after_dequeue = batch_size * 2
capacity = min_after_dequeue + 3 * batch_size
word_batch, feature_batch, label_batch = tf.train.shuffle_batch(
  [tokens, features, label], batch_size=batch_size, capacity=capacity,
  min_after_dequeue=min_after_dequeue)

# Start populating the filename queue.
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)


try:
    while not coord.should_stop():
        # Retrieve a single instance:
        t , f, l = sess.run([word_batch,feature_batch, label_batch])
        print t
        print f
        print l

except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')
finally:
    # When done, ask the threads to stop.
    coord.request_stop()

# Wait for threads to finish.
coord.join(threads)
sess.close()


[['finished' 'at' 'its' 'intraday' 'highs' '.' '<PAD>']
 ['w_-3' 'w_-2' 'w_-1' 'w_0' 'w_1' 'w_2' 'w_3']
 ['its' 'intraday' 'highs' '.' '<PAD>' '<PAD>' '<PAD>']
 ['dollar' 'finished' 'at' 'its' 'intraday' 'highs' '.']]
[[['LOWER']
  ['LOWER']
  ['LOWER']
  ['LOWER']
  ['LOWER']
  ['PUNCT']
  ['<PAD>']]

 [['cap_-3']
  ['cap_-2']
  ['cap_-1']
  ['cap_0']
  ['cap_1']
  ['cap_2']
  ['cap_3']]

 [['LOWER']
  ['LOWER']
  ['LOWER']
  ['PUNCT']
  ['<PAD>']
  ['<PAD>']
  ['<PAD>']]

 [['LOWER']
  ['LOWER']
  ['LOWER']
  ['LOWER']
  ['LOWER']
  ['LOWER']
  ['PUNCT']]]
['JJ' 'label' '.' 'PRP$']
[['<PAD>' '<PAD>' '<PAD>' 'the' 'dollar' 'finished' 'at']
 ['<PAD>' '<PAD>' 'and' 'while' 'the' 'defense' 'secretary']
 ['<PAD>' '<PAD>' 'the' 'dollar' 'finished' 'at' 'its']
 ['the' 'defense' 'secretary' 'is' 'speaking' 'out' ',']]
[[['<PAD>']
  ['<PAD>']
  ['<PAD>']
  ['TITLE']
  ['LOWER']
  ['LOWER']
  ['LOWER']]

 [['<PAD>']
  ['<PAD>']
  ['TITLE']
  ['LOWER']
  ['LOWER']
  ['LOWER']
  ['LOWER']]

 [['